In [260]:
import numpy as np
from cvxopt import matrix, solvers, spmatrix, sparse

In [344]:
def permutation (X,i):# return a column Z verified the hypothesis z_2>...z_f
    #X is a vector, i-th column of the Matrix C
    #
    C_ii=X[i]
    a=np.max(X)+1
    X[i]=a
    index=np.argsort(X)[::-1]
    Z=np.sort(X)[::-1]
    Z[0]=C_ii
    return Z, index, C_ii
    

def proj_01(x): #projection on interval [0,1]
    if x>1 :return 1
    if x<0 :return 0
    else :  return x
    
    
def proj_on_phi_0 (Z,index):#algo 5 
    f=Z.shape[0] 
    x=np.zeros(f)
    xx = np.zeros(f)
    mu = Z[0]
    for k in range(1,f): 
        if Z[k] <= proj_01(mu):
            k_c=k
            break
        else:
            mu=(k-1)/k*mu+1./k*Z[k]
    print 'k_c=',k_c,'mu=',mu
    x[0]=proj_01(mu)
    for k in range(1,k_c):
        x[k]=proj_01(mu)
    for k in range(k_c,f): 
        x[k]=max(Z[k],0)
    #print 'x=',x    
    #to return X, after permutation inverse
    for i in range(f):
        xx[index[i]]=x[i]
    return xx

def proj_on_phi_final(C):
    f = C.shape[0]
    for i in range(f):
        X = np.squeeze(np.asarray(C[:,i]))
        print X.shape
        Z, index, C_ii=permutation(X,i)
        #there always has a problem of compatibility between matrix, array of i-th column of C
        C[:,i]=np.matrix(proj_on_phi_0(Z,index)).T
        #C[:,i]=proj_on_phi_0(Z,index)

    return C
        

def Norm_inf1 (X,W):
    # X, W are two matrix of shape respectively f*n et r*n
    f,n = X.size
    r = W.size[0]
    
    print f,n,r
    
    P = matrix(W).trans()
    F = matrix(1.0,(f,r))
    
    onesn = matrix(1.0,(n,1))
    Idn = spmatrix(1.0, range(n),range(n))
    Idr = spmatrix(1.0, range(r),range(r))
    Zrn = spmatrix(0,[r-1],[n-1])
    Zr = spmatrix(0,[r-1],[0])
    #Zn = spmatrix(0,[n-1],[0])
    A = sparse([ [P,-P,-Idr], [-Idn,-Idn,Zrn] ]) 
    
    for i in range(f):
        V =  X[i,:].trans()
        C = matrix([[V,-V,Zr]])
        e = matrix([ [Zr, onesn] ])
        solution = solvers.lp(e,A,C)['x']
        F[i,:] = solution[range(r)].trans()
    return F    
    

def Hottpixx(X,s_p,s_d,N):
    f,n = X.shape
    p = np.arange(f)[::-1]
    C = np.zeros((f,f))
    beta = 0
    GDindex=np.arange(n)
    mu = np.zeros(f)
    for j in range(f):#μj is the number of nonzeros in row j divided by n.
        mu[j] = (np.sum(X[j,:] != 0)+0.0)/n
    for t in range(N):
        for i in range(n*10):
            k=random.choice(GDindex)
            C = C + s_p * np.matrix(np.sign(X[:,k]-np.dot(C,X[:,k]))).T.dot(np.matrix(X[:,k])) - s_d*np.diag(mu*(beta-p))  
            
        #print 'C.shape=',C.shape
        print 'C=',C
        proj_on_phi_final(C)
        
        beta = beta +s_d*(np.trace(C)-r)#we assume r is known, r is a global variable
        print t,'-th iteration is done'
    print C
    diag=np.diag(C)
    print 'diag=',diag
    index=np.argsort(diag)[::-1]
    I=index[:r]
    #print I
    W = X[I]
    
    #convert X,W into matrix type defined in cvxopt in order to apply the argmin function
    W1=matrix(W)
    X1=matrix(X)
    F = Norm_inf1(X1,W1)
    #convert again F into numpy matrix
    F1=np.array(F)
    return F1,W



In [345]:
from cvxopt import matrix
from AGKM import AGKM
from cvxopt import normal
import numpy as np
A = matrix([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], (2,3))
B = matrix ([5,6,7],(1,3))
print(A)
W1=np.matrix([[1,0,0.,0],[0,1,0,0]])
F1=np.matrix([[1,0],[.5,.5],[0.25,0.75],[0,1]])
Y = np.dot(F1,W1)
X = matrix(Y)

[ 1.00e+00  3.00e+00  5.00e+00]
[ 2.00e+00  4.00e+00  6.00e+00]



In [256]:
print Y

[[ 1.    0.    0.    0.  ]
 [ 0.5   0.5   0.    0.  ]
 [ 0.25  0.75  0.    0.  ]
 [ 0.    1.    0.    0.  ]]


In [352]:
r=2
A,B=Hottpixx(Y,0.1,0.001,100)

 C= [[ 0.48  0.45  0.45  0.45]
 [ 0.45  0.49  0.45  0.45]
 [ 0.45  0.45  0.47  0.45]
 [ 0.45  0.45  0.45  0.45]]
(4,)
k_c= 1 mu= 0.48
(4,)
k_c= 1 mu= 0.49
(4,)
k_c= 1 mu= 0.47
(4,)
k_c= 1 mu= 0.45
0 -th iteration is done
C= [[ 0.4350011  0.375      0.375      0.375    ]
 [ 0.375      0.4550022  0.375      0.375    ]
 [ 0.375      0.375      0.4150022  0.375    ]
 [ 0.375      0.375      0.375      0.3750011]]
(4,)
k_c= 1 mu= 0.4350011
(4,)
k_c= 1 mu= 0.4550022
(4,)
k_c= 1 mu= 0.4150022
(4,)
k_c= 1 mu= 0.3750011
1 -th iteration is done
C= [[ 0.4150054  0.325      0.325      0.325    ]
 [ 0.325      0.4450108  0.325      0.325    ]
 [ 0.325      0.325      0.3850108  0.325    ]
 [ 0.325      0.325      0.325      0.3250054]]
(4,)
k_c= 1 mu= 0.415005399934
(4,)
k_c= 1 mu= 0.445010799868
(4,)
k_c= 1 mu= 0.385010799868
(4,)
k_c= 1 mu= 0.325005399934
2 -th iteration is done
C= [[ 0.445014  0.325     0.325     0.325   ]
 [ 0.325     0.485028  0.325     0.325   ]
 [ 0.325     0.325     0.40502

In [353]:
print B

[[ 0.5  0.5  0.   0. ]
 [ 1.   0.   0.   0. ]]


In [201]:
a = np.array([1,4,3,2,-1,0.6,0.7,1.4])
#np.sort(a) 
#print np.append(a,5),np.max(a)

In [208]:
a = np.array([1,4,3,2,-1,0.6,0.7,1.4])

print a
Z,index,C_ii=permutation(a,1)

[ 1.   4.   3.   2.  -1.   0.6  0.7  1.4]


In [209]:
print Z,index,C_ii

[ 4.   3.   2.   1.4  1.   0.7  0.6 -1. ] [1 2 3 7 0 6 5 4] 4.0


In [220]:
proj_on_phi_0(Z,index)

k_c= 7 mu= 0.1


array([ 0.1,  0. ,  0.1,  0.1,  0. ,  0.1,  0.1,  0.1])

In [24]:
temp = np.random.randint(1,10, 10)
np.sort(temp)[::-1]

array([7, 7, 6, 6, 6, 6, 4, 3, 3, 1])

In [25]:
temp

array([6, 4, 6, 6, 7, 3, 1, 3, 7, 6])

In [73]:
max(-4,0)

0

In [75]:
np.arange(5)

array([0, 1, 2, 3, 4])

In [83]:
import random
my_choice = random.choice(np.arange(5))
print my_choice

3


In [228]:
X=np.array([[1,2,3,4],[3,4,5,6]])
XX=np.matrix([[1,2,3,4],[3,4,5,6]])
C=np.array([[5,1],[.4,.5]])
k=1
np.matrix(np.sign(X[:,k]-np.dot(C,X[:,k]))).T#.dot(np.matrix(X[:,k]))

matrix([[-1.],
        [ 1.]])

In [184]:
XX=np.zeros((3,3))


In [185]:
XX[:,2]

array([ 0.,  0.,  0.])

In [171]:
#X[1,:]=np.arange(4)
print X

[[1 2 3 4]
 [0 1 2 3]]


In [122]:
XX=np.diag(np.arange(10))

In [126]:
np.diag(XX)==4

array([False, False, False, False,  True, False, False, False, False, False], dtype=bool)

In [230]:
print X[:,1].shape,X[1,:].shape

(2,) (4,)


In [190]:
np.zeros((1,4)).shape

(1, 4)

In [233]:
M = np.matrix([[1], [2], [3], [4]])

In [237]:
M

matrix([[1],
        [2],
        [3],
        [4]])

In [238]:
np.asarray(M)

array([[1],
       [2],
       [3],
       [4]])

In [239]:
A = np.squeeze(np.asarray(M))

In [283]:
SPN = np.random.randint(-100, 100, ( 6, 6))
print SPN

[[ 54  84 -54 -94 -84 -98]
 [  0 -43 -21 -63 -26 -38]
 [ 30 -71  28 -56  23 -74]
 [  7  38 -42  88 -91 -76]
 [ 11 -69  36  -3 -96  82]
 [-36 -11  19 -91  92 -86]]


In [294]:
proj_on_phi_final(SPN)

(6,)
k_c= 2 mu= 1.0
x= [ 1.  1.  1.  1.  0.  0.]
(6,)
k_c= 2 mu= 1.0
x= [ 1.  1.  1.  0.  0.  0.]
(6,)
k_c= 2 mu= 1.0
x= [ 1.  1.  1.  0.  0.  0.]
(6,)
k_c= 1 mu= 0
x= [ 0.  0.  0.  0.  0.  0.]
(6,)
k_c= 2 mu= 1.0
x= [ 1.  1.  1.  0.  0.  0.]
(6,)
k_c= 2 mu= 1.0
x= [ 1.  1.  0.  0.  0.  0.]


array([[1, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0],
       [1, 1, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 1],
       [0, 0, 1, 0, 1, 1]])

In [320]:
XX=np.ones((5,5))

In [323]:
proj_on_phi_final(XX)

(5,)
k_c= 1 mu= 1.0
(5,)
k_c= 1 mu= 1.0
(5,)
k_c= 1 mu= 1.0
(5,)
k_c= 1 mu= 1.0
(5,)
k_c= 1 mu= 1.0


array([[ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.]])